In [270]:
class Atom:
    def __init__(self, name, variables):
        self.name = name
        self.variables = variables
        self.arity = len(variables)
        
    def replace_variables(self, variable_map):
        var_to_replace = []
        try:
            var_to_replace = list(filter(lambda var: var in variable_map, self.variables))
            actual_atom = Atom(self.name, [variable_map[var] if var in var_to_replace else var for var in self.variables])
            return actual_atom
        except KeyError:
            print("Wrong variables given to action.")
        
        
        
    def __eq__(self, other):
        return self.name == other.name and self.arity == other.arity and self.variables == other.variables
    
    def __str__(self):
        var_string = ""
        for var in self.variables:
            var_string += var
            var_string += ", "
        var_string = var_string[:-2]
        return "Atom: " + self.name +"(" + var_string + ")"

In [271]:
class State:
    def __init__(self, name, atoms):
        self.name = name
        self.atoms = atoms
        self.length = len(atoms)
    
    def removeAtom(self, atom):
        # if atom not in s then do nothing
        try:
            self.atoms.remove(atom)
            self.length -= 1
        except ValueError:
            pass
    
    def addAtom(self, atom):
        self.atoms.append(atom)
        self.length += 1
    
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return self.atoms == other.atoms
    
    def __str__(self):
        state_str = "State " + self.name + ": "
        for atom in self.atoms:
            state_str += str(atom)[6:] + "^"
        return state_str[:-1]
    
    def copy(self):
        return State(s.name, s.atoms.copy())
    

In [272]:
class Action:
    '''
        variables can be anything when defining the actions, they will be replaced by actual variables
        at applicability check or at execution of the action
    '''
    
    def __init__(self, name, preconditions, positive_effects, negative_effects, variables):
        self.name = name
        self.preconditions = preconditions
        self.positive_effects = positive_effects
        self.negative_effects = negative_effects
        self.variables = variables
    
    def isPracticalInState(self, s, variables):
        practical = True
        i = 0
        variable_map = self.createVariableMap(variables)
        while practical and i<len(self.preconditions):
            actual_atom = self.preconditions[i].replace_variables(variable_map)
            practical = practical and (actual_atom in s.atoms)
            i += 1 
        return practical
    
    def execute(self, s, variables):
        if self.isPracticalInState(s, variables):
            variable_map = self.createVariableMap(variables)

            for effect in self.negative_effects:
                s.removeAtom(effect.replace_variables(variable_map))

            for effect in self.positive_effects:
                s.addAtom(effect.replace_variables(variable_map))
        else:
            print("This action is not applicable here.")
            
    def createVariableMap(self, variables):
        variable_map = {}
        if len(variables) != len(self.variables):
            print("Wrong number of variables given. Please check action schema.")
            return None
       
        for i, var in enumerate(self.variables):
            variable_map[var] = variables[i]
            
        return variable_map
        

In [274]:
class LogicSolver:
    def __init__(self, initial_state, goal, actions):
        self.initial_state = initial_state
        self.goal = goal
        self.actions = actions
    
    def getPracticalActionsInState(self, s):
        practical_actions = []
        for action in self.actions:
            practical = True
            i = 0
            sorted_atoms = {}
            for atom in action.preconditions:
                sorted_atoms[atom.name] = []
            for atom in s.atoms:
                sorted_atoms[atom.name].append(atom.variables)
                
            variable_map = {}
            for var in action.variables:
                variable_map[var] = None
            
            while practical and i<len(action.preconditions):
                
                
                
                i+=1
        

In [275]:
p1 = Atom("On",["A", "1"])
p2 = Atom("On",["A", "2"])
b1 = Atom("Box", ["1"])
b2 = Atom("Box", ["2"])

In [276]:
s0 = State("s0", [p1, p2, b1, b2])
print(s0)

State s0: On(A, 1)^On(A, 2)^Box(1)^Box(2)


In [277]:
move = Action(name= "Move", 
              preconditions= [Atom("Box", ['x']), 
                              Atom("On", ["A","x"])],
              positive_effects = [Atom("On", ["B","x"])],
              negative_effects = [Atom("On", ["A","x"])], 
              variables = ["x"])

In [279]:
move.isPracticalInState(s0, ["3"])

False

In [281]:
move.execute(s0, ["2"])
print(s0)

State s0: Box(1)^Box(2)^On(B, 1)^On(B, 2)
